# MQTT Part 1: Subscribe and Publish



## MQTT Client for Python: Paho


Paho and is an [open-source Eclipse project](https://www.eclipse.org/paho/) na dprovides support for an MQTT client in Python. 

Just run the cell below to install the Python packages:

In [ ]:
!pip install --upgrade paho-mqtt

Read the output. If this command fails, use the pip command on your command line.

    pip install --upgrade paho-mqtt

The [API docs of Paho are here](https://pypi.python.org/pypi/paho-mqtt).

We then need to import the package:

In [1]:
import paho.mqtt.client as mqtt

## MQTT Broker

For the communication via MQTT, you need an MQTT broker. Ideally, you should start your own MQTT broker so you have control over everything. 

For some testing and prototyping, you can also use some of the open MQTT brokers from various projects. Some of them are listed here:

https://github.com/mqtt/mqtt.github.io/wiki/public_brokers

Of course, never send any sensitive information via these servers.

We decide to use the broker from the Eclipse IoT project. (We assign here borker address and port in the same line.)

In [2]:
broker, port = 'iot.eclipse.org', 1883

# settings for a local MQTT broker on the same machine:
#broker, port = 'localhost', 1883

## Our Own MQTT Client in Python

We now want to create our own MQTT client. What we want to achieve is the following:

 * Connect to one of the public MQTT brokers.
 * Subscribe to the topic #.
 * List the incoming messages.
 * Disconnect after we received 20 messages.
 
The class below contains the code we need. We deciced to declare this as a Python class, since we want to have a variable `count` that counts the number of incoming messages, for then disconnecting after we received 20. Declaring a class gives us a good way to share the `count` variable. 

In [5]:
from threading import Thread

class MQTT_Client_1:
    
    def __init__(self):
        self.count = 0
        print(self.count)
        
    def on_connect(self, client, userdata, flags, rc):
        print('on_connect(): {}'.format(mqtt.connack_string(rc)))
        
    def on_message(self, client, userdata, msg):
        print('on_message(): topic: {}'.format(msg.topic))
        self.count = self.count + 1
        if self.count == 20:
            self.client.disconnect()
            print('disconnected after 20 messages')
        
    def start(self, broker, port):
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        print('Connecting to {}:{}'.format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("#")
        try:
            thread = Thread(target=self.client.loop_forever())
            thread.start()
        except KeyboardInterrupt:
            print('Interrupted')
            self.client.disconnect()

Above we declare a class `MQTT_Client_1` that encapsulates the Paho MQTT client with our own logic. 

 * The method `__init__()` is called when we construct an instance of the class. We use it here to set our counter to 0. Note that we use `self.count` to declare this variable, so it is available also in the other methods.
 * The method `on_connect()` is called by the Paho client once a connection is established. We just print out a decoded form of the message we receive back to be sure about what is going on.
 * The method `on_message()` will be called by the Paho client every time we receive an MQTT message. Since we want to disconnect after the first 20 messages, we increase the counter in this method and also have the code for disconnection.
 * The method `start()` creates the client an connect to the broker. 
   * It also subscribes to the topic. 
   * In the end, it starts a thread that will run the client forever. 
   * We place this into a thread so that the notebook still stays responsive and prints on the received messages.
   * We also catch any KeyInterrupt error, in case you interrupt a notebook cell, so that we disconnect properly.

In [6]:
myclient = MQTT_Client_1()
myclient.start(broker, port)

0
Connecting to iot.eclipse.org:1883
on_connect(): Connection Accepted.
on_message(): topic: /DefendIt/PZEM
on_message(): topic: /DefendIt/SanIsidro
on_message(): topic: /topic/12
on_message(): topic: /topic/level
on_message(): topic: /topic/qos0
on_message(): topic: /topic/will
on_message(): topic: /events/testevent
on_message(): topic: /034L2Xe68z/001/wds/door
on_message(): topic: /034L2Xe68z/001/wds/door/age
on_message(): topic: /034L2Xe68z/001/wds/door/open
on_message(): topic: /034L2Xe68z/001/wds/motion
on_message(): topic: /034L2Xe68z/001/wds/motion/age
on_message(): topic: /034L2Xe68z/001/wds/motion/perc
on_message(): topic: /034L2Xe68z/001/wds/temp
on_message(): topic: /034L2Xe68z/001/wds/lumi
on_message(): topic: /034L2Xe68z/000/time
on_message(): topic: /noderedpi/state/online
on_message(): topic: /30c216b0ed/1/door
on_message(): topic: /30c216b0ed/1/motion
on_message(): topic: /30c216b0ed/1/temp
disconnected after 5 messages
on_message(): topic: /30c216b0ed/1/lumi


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'int' object is not callable



In [ ]:
!pip install --upgrade paho-mqtt

In [ ]:
import sys
sys.executable

In [ ]:
class Frank_MQTT_Client:
    
    def __init__(self):
        self.count = 0
        print(self.count)
        
    def on_connect(self, client, userdata, flags, rc):
        print('on_connect(): {}'.format(mqtt.connack_string(rc)))
        
    def on_message(self, client, userdata, msg):
        print('on_message(): topic: {}'.format(msg.topic))
        self.count = self.count + 1
        if self.count > 5:
            self.client.disconnect()
            print('disconnected')
        
    def start(self, broker, port):
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        print('Connecting to {}:{}'.format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("/#")
        try:
            thread = Thread(target=self.client.loop_forever())
            thread.start()
        except KeyboardInterrupt:
            print('Interrupted')
            self.client.disconnect()
        
        
frank = Frank_MQTT_Client()
frank.start('iot.eclipse.org', 1883)